In [1]:
import qiskit
from qiskit.quantum_info import state_fidelity
import numpy as np
from numpy import linalg as LA
import qib
import matplotlib.pyplot as plt
import scipy
import h5py


I2 = np.eye(2)
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])
ket_0 = np.array([[1],[0]])
ket_1 = np.array([[0],[1]])
rho_0_anc = ket_0 @ ket_0.T
rho_1_anc = ket_1 @ ket_1.T

In [23]:
import sys
sys.path.append("../../src/brickwall_ansatz")
from optimize import optimize
from utils import construct_heisenberg_local_term
from ansatz import ansatz
import rqcopt as oc
from qiskit.quantum_info import random_statevector
from scipy.sparse.linalg import expm_multiply
from scipy.linalg import expm

L = 6
# construct Hamiltonian
latt = qib.lattice.IntegerLattice((L,), pbc=True)
field = qib.field.Field(qib.field.ParticleType.QUBIT, latt)
J = (1, 1, 1)
h = (3, -1, 1)
hamil = qib.HeisenbergHamiltonian(field, J, h).as_matrix().toarray()
t = 0.25
r = 1

hloc1 = construct_heisenberg_local_term((J[0], 0   ,    0), (0, h[1], 0))
hloc2 = construct_heisenberg_local_term((0   , J[1],    0), (0, 0, h[2]))
hloc3 = construct_heisenberg_local_term((0   , 0   , J[2]), (h[0], 0, 0))
hlocs = (hloc1, hloc2, hloc3)
Vs = [scipy.linalg.expm(-1j*t*hlocs[0]/r), scipy.linalg.expm(-1j*t*hlocs[1]/r), 
      scipy.linalg.expm(-1j*t*hlocs[2]), 
      #scipy.linalg.expm(-1j*t*hlocs[1]/r), scipy.linalg.expm(-1j*t*hlocs[0]/r)
]
cs = [np.kron(X,  Z), np.kron(Z@Y, I2), np.kron(X@Z, I2), np.kron(Y, Z)]
#cs = [np.eye(4) for i in range(4)]

Vlist_start = [cs[0], Vs[0], Vs[0], cs[1], Vs[1], Vs[1], cs[2], Vs[2], Vs[2], cs[3]]
Vlist_reduced = [Vs[0], Vs[0], Vs[1], Vs[1], Vs[2], Vs[2]]
control_layers = [0, 3, 6, 9] # 4 control layers

perms_extended = [[i for i in range(L)] if i%2==0 else [i for i in range(1, L)]+[0] for i in range(len(Vlist_start))]
perms_ext_reduced = []
for i, perm in enumerate(perms_extended):
    if i not in control_layers:
        perms_ext_reduced.append(perm)

state = random_statevector(2**L).data
print(np.linalg.norm(ansatz(Vlist_start, L, perms_extended) - expm(
    1j * t * hamil), ord=2))
print(np.linalg.norm(ansatz(Vlist_reduced, L, perms_ext_reduced) - expm(
    -1j * t * hamil), ord=2))


1.8364885096249233
1.912062497620445


In [24]:
eta, gamma = (3, 2)
niter = 50
Vlist, f_iter, err_iter = optimize(L, expm(-1j * t * hamil), eta, gamma, Vlist_start, perms_extended, niter=niter)
plt.plot(err_iter)
plt.yscale('log')
print(err_iter[-1])

Conv tol.  1e-10
err:  0.6650289938989558
err:  0.658314432703654
err:  0.6452221470210376
err:  0.6205412164243485
err:  0.5783782572573429
err:  0.5414530889462718
err:  0.5208625646216185
err:  0.5127424583063033
err:  0.5060040177449454
err:  0.5023112642226897
err:  0.5006038551588645
err:  0.4997195532358474
err:  0.49880560967908927
err:  0.4977777438240858
err:  0.4963877139081374
err:  0.4947969118914243
err:  0.4926996166126332


KeyboardInterrupt: 

In [21]:
r = 2
Vs = [scipy.linalg.expm(-1j*t*hlocs[0]/r), scipy.linalg.expm(-1j*t*hlocs[1]/r), 
      scipy.linalg.expm(-1j*t*hlocs[2]), 
      scipy.linalg.expm(-1j*t*hlocs[1]/r), scipy.linalg.expm(-1j*t*hlocs[0]/r)
]
cs = [np.eye(4) for i in range(4)]
Vlist_start = [cs[0], Vs[0], Vs[0], Vs[1], Vs[1], cs[1], Vs[2], Vs[2], Vs[1], Vs[1], cs[2], Vs[0], Vs[0], np.eye(4), np.eye(4), cs[3]]
Vlist_reduced = [Vs[0], Vs[0], Vs[1], Vs[1], 
                 Vs[2], Vs[2], Vs[1], Vs[1], 
                 Vs[0], Vs[0], np.eye(4), np.eye(4)]
control_layers = [0, 5, 10, 15] # 4 control layers

perms_extended = [[i for i in range(L)] if i%2==0 else [i for i in range(1, L)]+[0] for i in range(len(Vlist_start))]
perms_ext_reduced = []
for i, perm in enumerate(perms_extended):
    if i not in control_layers:
        perms_ext_reduced.append(perm)

eta, gamma = (3, 4)
niter = 50
Vlist_ID, f_iter, err_iter = optimize(L, expm(-1j * t * hamil), eta, gamma, Vlist_ID, perms_extended, niter=niter)
plt.plot(err_iter)
plt.yscale('log')
print(err_iter[-1])

Conv tol.  1e-10
err:  0.002882588662371788
err:  0.0028499556420304994
err:  0.0028213240459808997
err:  0.0027786203239670293
err:  0.0026715444869597205
err:  0.002501208618120865
err:  0.0023454794141790636
err:  0.002184581870500746
err:  0.0020105527568193837
err:  0.0018486644683815001
err:  0.0016838289701591735
err:  0.001523220662733138
err:  0.0013627182103530533
err:  0.0012085203681304613
err:  0.001060921047567831
err:  0.0009152495643097724
err:  0.0007877563380358499
err:  0.0006819161773494731
err:  0.0005928719399727056
err:  0.0005296887517399762
err:  0.00047338062755875043
err:  0.00043024600215268993
err:  0.00038364360485088067
err:  0.0003422437118232313
err:  0.00029821704870158783
err:  0.0002565609144864478
err:  0.00022046896273630878
err:  0.00018070006293013918
err:  0.000147965042623599
err:  0.0001175615892091697
err:  9.245919210087994e-05
err:  7.281612033671614e-05
err:  5.6743772802514414e-05
err:  4.335836880953181e-05
err:  3.4630091345966285e-05
e

KeyboardInterrupt: 

In [14]:
"""
    Test operations of Compressed Time Evolution.
"""

L = 10
# construct Hamiltonian
latt = qib.lattice.IntegerLattice((L,), pbc=True)
field = qib.field.Field(qib.field.ParticleType.QUBIT, latt)
J = (1, 1, 1)
h = (3, -1, 1)
hamil = qib.HeisenbergHamiltonian(field, J, h).as_matrix()
t = 0.125
perms_extended = [[[i for i in range(L)]] if i%2==0 else [[i for i in range(1, L)]+[0]] for i in range(len(Vlist_start))]
perms_ext_reduced = []
for i, perm in enumerate(perms_extended):
    if i not in control_layers:
        perms_ext_reduced.append(perm)


state = random_statevector(2**L).data
print("state fidelity error of the optimized point: ", 1 - state_fidelity(ansatz_sparse(Vlist_reduced, L, perms_ext_reduced, state), expm_multiply(
    -1j * t * hamil, state)))


state fidelity error of the optimized point:  7.737679507568984e-06
